In [1]:
import pandas as pd
import warnings

warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

from hypex.dataset import Dataset, ExperimentData, InfoRole, TreatmentRole, TargetRole
from hypex.analyzers.aa2 import AAScoreAnalyzer
from hypex.experiments.aa2 import AATest

C:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ImportError: cannot import name 'ABAnalyzer' from partially initialized module 'hypex.analyzers' (most likely due to a circular import) (C:\Users\User\PycharmProjects\HypEx\hypex\analyzers\__init__.py)

In [ ]:
data = Dataset(
    roles={
        "user_id": InfoRole(int),
        "treat": TreatmentRole(int),
        "pre_spends": TargetRole(),
        "post_spends": TargetRole(),
        "gender": TargetRole()
    }, data="data.csv",
)
# data

In [ ]:
aa = AATest(range(10))
res = aa.execute(ExperimentData(data))
res

In [ ]:
res = AAScoreAnalyzer().execute(res)
res.analysis_tables

In [ ]:
res.analysis_tables['AATest┴┴']['post_spends┴KSTest┴p-value┴test'].data.iloc[:, 0].hist()

In [ ]:
res.analysis_tables['AATest┴┴'].iloc[0, 0]